# Context
Phase one of this project is feature extraction.\
This notebook drives the feature extraction process.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

In [2]:
# Import project-wide and PH1 specific variables and functions
import superheader as sup
import PH1header as ph1

I0000 00:00:1750115615.838135 4792778 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1750115615.844610 4792907 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750115615.850647 4792909 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1750115615.852682 4792778 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1750115615.921375 4792916 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750115615.928799 4792916 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for fe

# Local filesystem to pandas dataframe
The first step we take is to read the data from our filesystem into a pandas dataframe.
We use nltk to help us in this process.

In [3]:
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
import pandas as pd

## NLTK Corpus

In [4]:
RAWcorpus = CategorizedPlaintextCorpusReader(sup.RAW_DATA_ROOT, fileids=rf".*_({sup.CLASSES_REGEX_GROUP})\.mp4$", cat_pattern=r"(.*)/")

## Pandas DataFrame

In [5]:
RAWlist = []

for fileid in RAWcorpus.fileids():
  RAWlist.append(ph1.get_tags(fileid))

RAWdf = pd.DataFrame(RAWlist, columns=sup.tag_columns+[sup.class_numeric_column])

In [6]:
RAWdf

,fileid,person_id,cycle_num,handedness,class_name,class_numeric
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_0.mp4,p01,1,0,0,37
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_10.mp4,p01,1,0,10,36
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_2.mp4,p01,1,0,2,29
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_3.mp4,p01,1,0,3,30
...,...,...,...,...,...,...
4151,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_v.mp4,p11,5,1,v,23
4152,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_w.mp4,p11,5,1,w,24
4153,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_x.mp4,p11,5,1,x,25
4154,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_y.mp4,p11,5,1,y,26


In [7]:
RAWdf.groupby(by='class_name').count()

,fileid,person_id,cycle_num,handedness,class_numeric
class_name,,,,,
0,30,30,30,30,30
1,108,108,108,108,108
10,108,108,108,108,108
2,108,108,108,108,108
3,108,108,108,108,108
4,108,108,108,108,108
5,108,108,108,108,108
6,108,108,108,108,108
7,108,108,108,108,108


### Selecting relevant videos and classes

#### Counting the number of frames per video
Dropping videos with less than 12 frames, as often a majority of the frames are not an accurate representation of the sign

In [8]:
RAWdf["frame_count"] = RAWdf[sup.fileid_col].apply(ph1.step1.count_frames)
count = (RAWdf["frame_count"]< 12).sum()
print(count)

114


In [9]:
RAWdf = RAWdf[RAWdf["frame_count"] >= 12].reset_index(drop=True)  # drops rows and cleans the index
RAWdf

,fileid,person_id,cycle_num,handedness,class_name,class_numeric,frame_count
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_0.mp4,p01,1,0,0,37,22
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,36
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_10.mp4,p01,1,0,10,36,34
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_2.mp4,p01,1,0,2,29,22
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_3.mp4,p01,1,0,3,30,34
...,...,...,...,...,...,...,...
4037,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_v.mp4,p11,5,1,v,23,32
4038,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_w.mp4,p11,5,1,w,24,28
4039,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_x.mp4,p11,5,1,x,25,45
4040,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_y.mp4,p11,5,1,y,26,39


In [10]:
RAWdf = RAWdf.drop(
    columns=[
        "frame_count",
    ]
)

#### Selecting relevant classes
The class corresponding to '0' is much smaller than the rest, so we will drop
all of those samples

In [11]:
len(RAWdf[sup.class_name_column].value_counts())


39

In [12]:
RAWdf = RAWdf[RAWdf[sup.class_name_column] != '0'].reset_index(drop=True)
RAWdf

,fileid,person_id,cycle_num,handedness,class_name,class_numeric
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_10.mp4,p01,1,0,10,36
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_2.mp4,p01,1,0,2,29
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_3.mp4,p01,1,0,3,30
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_4.mp4,p01,1,0,4,31
...,...,...,...,...,...,...
4007,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_v.mp4,p11,5,1,v,23
4008,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_w.mp4,p11,5,1,w,24
4009,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_x.mp4,p11,5,1,x,25
4010,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_y.mp4,p11,5,1,y,26


The signs for "7" and "G" are visually indifferentiable in this dialect of MSL.

Normally it is obvious with context which sign is being used, but because of the limitations of this project and the fact we are doing "Isolated Sign Language Recognition", it is not possible to differentiate.

Therefore, to avoid class imbalance we are dropping the videos for 7

In [13]:
RAWdf = RAWdf[RAWdf[sup.class_name_column] != '7'].reset_index(drop=True)
RAWdf

,fileid,person_id,cycle_num,handedness,class_name,class_numeric
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_10.mp4,p01,1,0,10,36
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_2.mp4,p01,1,0,2,29
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_3.mp4,p01,1,0,3,30
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_4.mp4,p01,1,0,4,31
...,...,...,...,...,...,...
3899,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_v.mp4,p11,5,1,v,23
3900,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_w.mp4,p11,5,1,w,24
3901,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_x.mp4,p11,5,1,x,25
3902,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_y.mp4,p11,5,1,y,26


# Video to landmarks

## Per frame data

In [14]:
PH1_per_frame_df = pd.concat(RAWdf.apply(ph1.step1.extract_landmarks_per_frame, axis=1).tolist(), ignore_index=True)
PH1_per_frame_df

W0000 00:00:1750115703.877696 4792918 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


,fileid,person_id,cycle_num,handedness,class_name,class_numeric,first_frame,current_frame,num_candidate_hands,current_candidate_hand,...,current_candidate_pose,p0x,p0y,p0z,p11x,p11y,p11z,p12x,p12y,p12z
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,0,...,0,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,2,...,0,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,0,...,0,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,2,...,0,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,2,4,0,...,0,0.497244,0.318666,-0.457492,0.579072,0.523931,-0.151225,0.423043,0.499277,-0.169967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91790,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,9,4,2,...,0,0.484490,0.289181,-0.421546,0.572918,0.474172,-0.078406,0.397896,0.484275,-0.113990
91791,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,0,...,0,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788
91792,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,2,...,0,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788
91793,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,11,4,0,...,0,0.480457,0.287692,-0.410172,0.570401,0.466859,-0.068966,0.398920,0.484171,-0.099403


In [15]:
# There should never be more than one candidate pose, since there is only one
# person per video in our dataset
# This will make our model "weaker" during inference in the sense that we can't
# train it to focus on the person signing like we may be able to train it to 
# focus on the active hand.
PH1_per_frame_df["num_candidate_poses"].ne(1).sum()

0

## Active hand
As of this point in the process, for each frame, it is possible that mediapipe
detects multiple candidate hands. This means that if we wish to store the 
landmark data video by video in a single row contain the landmark data for each
of the video's frames, we have one of two choices:
1. We store all the data detected in each frame, including that for inactive
hands
2. We choose only the likeliest active hand and store the data for it

The purpose of this project is to train a model which predicts the sign being
performed. Since all of our signs are single-hand signs, this means the
information from all other hands has to be discarded. Therefore, we choose
option 2.

For each video (and thus for each frame in each video), we know the expected 
handedness thanks to the naming convention for the files used by the creator of
the dataset. Also, mediapipe gives us confidence scores for each of the hands
it detects. Therefore, we will use this information to go through our perframe
dataset, and for frames for which there is more than one hand detected, choose
to keep only the data for the hand that is the likeliest to be active. 


### Frames with only one detected hand

In [16]:
PH1_only_one_df = PH1_per_frame_df.copy()
PH1_only_one_df[sup.active_hand_col] = -1

In [17]:
pair_counts = PH1_only_one_df.groupby([sup.fileid_col, sup.current_frame_col]).size()
pair_counts


fileid                                             current_frame
p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4      0                2
                                                   1                2
                                                   2                2
                                                   3                2
                                                   4                2
                                                                   ..
p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4  7                2
                                                   8                2
                                                   9                2
                                                   10               2
                                                   11               2
Length: 46844, dtype: int64

In [18]:
only_one_detected_hand_mask = PH1_only_one_df.set_index([sup.fileid_col, sup.current_frame_col]).index.map(pair_counts) == 1
print(only_one_detected_hand_mask.sum())


1893


In [19]:
PH1_only_one_df.loc[only_one_detected_hand_mask, sup.active_hand_col] = 1
PH1_only_one_df

,fileid,person_id,cycle_num,handedness,class_name,class_numeric,first_frame,current_frame,num_candidate_hands,current_candidate_hand,...,p0x,p0y,p0z,p11x,p11y,p11z,p12x,p12y,p12z,active_hand
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,0,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,-1
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,2,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,-1
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,0,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,-1
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,2,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,-1
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,2,4,0,...,0.497244,0.318666,-0.457492,0.579072,0.523931,-0.151225,0.423043,0.499277,-0.169967,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91790,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,9,4,2,...,0.484490,0.289181,-0.421546,0.572918,0.474172,-0.078406,0.397896,0.484275,-0.113990,-1
91791,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,0,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,-1
91792,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,2,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,-1
91793,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,11,4,0,...,0.480457,0.287692,-0.410172,0.570401,0.466859,-0.068966,0.398920,0.484171,-0.099403,-1


In [20]:
count_minus_ones = (PH1_only_one_df[sup.active_hand_col] == -1).sum()
print(count_minus_ones)

89902


In [21]:
PH1_only_one_df.groupby('fileid').size()

fileid
p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4        24
p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_10.mp4       24
p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_2.mp4        24
p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_3.mp4        24
p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_4.mp4        24
                                                     ..
p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_v.mp4    24
p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_w.mp4    24
p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_x.mp4    24
p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_y.mp4    24
p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4    24
Length: 3904, dtype: int64

### Frames with only with one detected hand for which the handedness matches the expected handedness

In [22]:
PH1_check_handedness_df = PH1_only_one_df.copy()

In [23]:
def keep_correct_handedness(group):
    # Only proceed if group has more than 1 row
    if len(group) > 1:
        # Boolean mask where handedness matches detected_handedness
        matches = group[sup.handedness_column] == group["detected_handedness"]
        count_matches = matches.sum()
        
        if count_matches == 1:
            # Set active_hand: 1 for the matching row, 0 for the rest
            group.loc[matches, sup.active_hand_col] = 1
            group.loc[~matches, sup.active_hand_col] = 0
        # Else, do nothing (leave active_hand as is)
    return group

PH1_check_handedness_df = PH1_check_handedness_df.groupby(
    [sup.fileid_col, sup.current_frame_col], group_keys=False
).apply(keep_correct_handedness)


/var/folders/y5/xympzvrs537ffwjygt2x7wnh0000gn/T/ipykernel_23812/3534347682.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(keep_correct_handedness)


In [24]:
PH1_check_handedness_df

,fileid,person_id,cycle_num,handedness,class_name,class_numeric,first_frame,current_frame,num_candidate_hands,current_candidate_hand,...,p0x,p0y,p0z,p11x,p11y,p11z,p12x,p12y,p12z,active_hand
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,0,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,1
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,2,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,0
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,0,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,1
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,2,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,0
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,2,4,0,...,0.497244,0.318666,-0.457492,0.579072,0.523931,-0.151225,0.423043,0.499277,-0.169967,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91790,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,9,4,2,...,0.484490,0.289181,-0.421546,0.572918,0.474172,-0.078406,0.397896,0.484275,-0.113990,1
91791,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,0,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,1
91792,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,2,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,0
91793,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,11,4,0,...,0.480457,0.287692,-0.410172,0.570401,0.466859,-0.068966,0.398920,0.484171,-0.099403,1


In [25]:
count_minus_ones = (PH1_check_handedness_df[sup.active_hand_col] == -1).sum()
print(count_minus_ones)

1540


### Frames with multiple hands with the expected handedness

In [26]:
PH1_check_confidence_df = PH1_check_handedness_df.copy()

In [27]:
def keep_confident(group):
    if (group[sup.active_hand_col] == -1).all() and len(group) > 1:
        idx_max = group["confidence"].idxmax()
        group = group.copy()  # avoid potential SettingWithCopy issues
        group.loc[group.index == idx_max, sup.active_hand_col] = 1
        group.loc[group.index != idx_max, sup.active_hand_col] = 0

        best_confidence = group.loc[idx_max, "confidence"]
        if best_confidence < 0.9:
            print(f"not confident about anybody in f{group[sup.current_frame_col]}, {group[sup.fileid_col]}")
    return group

PH1_check_confidence_df = PH1_check_confidence_df.groupby(
    [sup.fileid_col, sup.current_frame_col], group_keys=False
).apply(keep_confident)



not confident about anybody in f26805    2
26806    2
Name: current_frame, dtype: int64, 26805    p04/Ciclo_1_5_Izquierda/Ciclo_1_5_Izquierda_10...
26806    p04/Ciclo_1_5_Izquierda/Ciclo_1_5_Izquierda_10...
Name: fileid, dtype: object
not confident about anybody in f26809    4
26810    4
Name: current_frame, dtype: int64, 26809    p04/Ciclo_1_5_Izquierda/Ciclo_1_5_Izquierda_10...
26810    p04/Ciclo_1_5_Izquierda/Ciclo_1_5_Izquierda_10...
Name: fileid, dtype: object
not confident about anybody in f26811    5
26812    5
Name: current_frame, dtype: int64, 26811    p04/Ciclo_1_5_Izquierda/Ciclo_1_5_Izquierda_10...
26812    p04/Ciclo_1_5_Izquierda/Ciclo_1_5_Izquierda_10...
Name: fileid, dtype: object
not confident about anybody in f28495    0
28496    0
Name: current_frame, dtype: int64, 28495    p04/Ciclo_2_5_Izquierda/Ciclo_2_5_Izquierda_10...
28496    p04/Ciclo_2_5_Izquierda/Ciclo_2_5_Izquierda_10...
Name: fileid, dtype: object
not confident about anybody in f28497    1
28498    1
Name: 

/var/folders/y5/xympzvrs537ffwjygt2x7wnh0000gn/T/ipykernel_23812/1759615024.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(keep_confident)


In [28]:
PH1_check_confidence_df

,fileid,person_id,cycle_num,handedness,class_name,class_numeric,first_frame,current_frame,num_candidate_hands,current_candidate_hand,...,p0x,p0y,p0z,p11x,p11y,p11z,p12x,p12y,p12z,active_hand
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,0,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,1
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,2,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,0
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,0,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,1
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,2,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,0
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,2,4,0,...,0.497244,0.318666,-0.457492,0.579072,0.523931,-0.151225,0.423043,0.499277,-0.169967,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91790,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,9,4,2,...,0.484490,0.289181,-0.421546,0.572918,0.474172,-0.078406,0.397896,0.484275,-0.113990,1
91791,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,0,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,1
91792,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,2,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,0
91793,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,11,4,0,...,0.480457,0.287692,-0.410172,0.570401,0.466859,-0.068966,0.398920,0.484171,-0.099403,1


In [29]:
count_minus_ones = (PH1_check_confidence_df[sup.active_hand_col] == -1).sum()
print(count_minus_ones)

0


In [30]:
print(PH1_check_confidence_df.columns.tolist())

['fileid', 'person_id', 'cycle_num', 'handedness', 'class_name', 'class_numeric', 'first_frame', 'current_frame', 'num_candidate_hands', 'current_candidate_hand', 'detected_handedness', 'confidence', 'h0x', 'h0y', 'h0z', 'h1x', 'h1y', 'h1z', 'h2x', 'h2y', 'h2z', 'h3x', 'h3y', 'h3z', 'h4x', 'h4y', 'h4z', 'h5x', 'h5y', 'h5z', 'h6x', 'h6y', 'h6z', 'h7x', 'h7y', 'h7z', 'h8x', 'h8y', 'h8z', 'h9x', 'h9y', 'h9z', 'h10x', 'h10y', 'h10z', 'h11x', 'h11y', 'h11z', 'h12x', 'h12y', 'h12z', 'h13x', 'h13y', 'h13z', 'h14x', 'h14y', 'h14z', 'h15x', 'h15y', 'h15z', 'h16x', 'h16y', 'h16z', 'h17x', 'h17y', 'h17z', 'h18x', 'h18y', 'h18z', 'h19x', 'h19y', 'h19z', 'h20x', 'h20y', 'h20z', 'num_candidate_poses', 'current_candidate_pose', 'p0x', 'p0y', 'p0z', 'p11x', 'p11y', 'p11z', 'p12x', 'p12y', 'p12z', 'active_hand']


### Dropping videos with missing frames

In [31]:
expected_frames = list(range(12))

missing_frames_grouped = PH1_check_confidence_df[PH1_check_confidence_df[sup.active_hand_col] == 1]\
  .groupby(sup.fileid_col)\
  .filter(lambda g: len(g) != 12)\
  .groupby(sup.fileid_col)

for fileid, group in missing_frames_grouped:
    actual_frames = group[sup.current_frame_col].tolist()
    missing_frames = sorted(set(expected_frames) - set(actual_frames))
    print(f"\nFileID: {fileid}")
    print(f"Missing frames: {missing_frames}")


FileID: p01/Ciclo_3_5_Izquierda/Ciclo_3_5_Izquierda_n.mp4
Missing frames: [10]

FileID: p04/Ciclo_3_5_Derecha/Ciclo_3_5_Derecha_q.mp4
Missing frames: [1, 3]

FileID: p04/Ciclo_5_5_Derecha/Ciclo_5_5_Derecha_k.mp4
Missing frames: [0]


In [32]:
for fileid, group in missing_frames_grouped:
  print(f"\nFileID: {fileid}")
  print(PH1_check_confidence_df[
    (PH1_check_confidence_df[sup.fileid_col] == fileid)]
    [[sup.current_frame_col, sup.active_hand_col]])


FileID: p01/Ciclo_3_5_Izquierda/Ciclo_3_5_Izquierda_n.mp4
      current_frame  active_hand
4567              0            1
4568              1            1
4569              2            1
4570              3            1
4571              4            1
4572              5            1
4573              6            1
4574              7            1
4575              8            1
4576              9            1
4577             11            1

FileID: p04/Ciclo_3_5_Derecha/Ciclo_3_5_Derecha_q.mp4
       current_frame  active_hand
29847              0            1
29848              2            1
29849              4            1
29850              5            1
29851              6            1
29852              6            0
29853              7            1
29854              8            1
29855              8            0
29856              9            0
29857              9            1
29858             10            1
29859             11            1

FileID: p04/C

In [33]:
PH1_no_missing_frames_df = PH1_check_confidence_df[
  ~PH1_check_confidence_df[sup.fileid_col].isin(
    [fileid for fileid, group in missing_frames_grouped]
  )
]

In [34]:
PH1_no_missing_frames_df

,fileid,person_id,cycle_num,handedness,class_name,class_numeric,first_frame,current_frame,num_candidate_hands,current_candidate_hand,...,p0x,p0y,p0z,p11x,p11y,p11z,p12x,p12y,p12z,active_hand
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,0,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,1
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,0,4,2,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,0
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,0,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,1
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,1,4,2,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,0
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,12,2,4,0,...,0.497244,0.318666,-0.457492,0.579072,0.523931,-0.151225,0.423043,0.499277,-0.169967,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91790,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,9,4,2,...,0.484490,0.289181,-0.421546,0.572918,0.474172,-0.078406,0.397896,0.484275,-0.113990,1
91791,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,0,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,1
91792,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,10,4,2,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,0
91793,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,27,11,4,0,...,0.480457,0.287692,-0.410172,0.570401,0.466859,-0.068966,0.398920,0.484171,-0.099403,1


# Write out

In [35]:
# These columns were only useful to determine the active hand.
PH1_final_df = PH1_no_missing_frames_df.drop(
    columns=[
        "num_candidate_hands",
        "current_candidate_hand",
        "detected_handedness",
        "confidence",
        "num_candidate_poses",
        "current_candidate_pose",
        "first_frame"
    ]
)

In [36]:
print(PH1_final_df.columns.tolist())

['fileid', 'person_id', 'cycle_num', 'handedness', 'class_name', 'class_numeric', 'current_frame', 'h0x', 'h0y', 'h0z', 'h1x', 'h1y', 'h1z', 'h2x', 'h2y', 'h2z', 'h3x', 'h3y', 'h3z', 'h4x', 'h4y', 'h4z', 'h5x', 'h5y', 'h5z', 'h6x', 'h6y', 'h6z', 'h7x', 'h7y', 'h7z', 'h8x', 'h8y', 'h8z', 'h9x', 'h9y', 'h9z', 'h10x', 'h10y', 'h10z', 'h11x', 'h11y', 'h11z', 'h12x', 'h12y', 'h12z', 'h13x', 'h13y', 'h13z', 'h14x', 'h14y', 'h14z', 'h15x', 'h15y', 'h15z', 'h16x', 'h16y', 'h16z', 'h17x', 'h17y', 'h17z', 'h18x', 'h18y', 'h18z', 'h19x', 'h19y', 'h19z', 'h20x', 'h20y', 'h20z', 'p0x', 'p0y', 'p0z', 'p11x', 'p11y', 'p11z', 'p12x', 'p12y', 'p12z', 'active_hand']


In [37]:
PH1_final_df

,fileid,person_id,cycle_num,handedness,class_name,class_numeric,current_frame,h0x,h0y,h0z,...,p0x,p0y,p0z,p11x,p11y,p11z,p12x,p12y,p12z,active_hand
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,0,0.413438,0.516854,-9.053193e-08,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,1
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,0,0.568475,0.911143,1.136299e-09,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,0
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,1,0.411601,0.525609,-9.427173e-08,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,1
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,1,0.568473,0.911162,1.245373e-09,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,0
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,2,0.409486,0.530432,-8.504437e-08,...,0.497244,0.318666,-0.457492,0.579072,0.523931,-0.151225,0.423043,0.499277,-0.169967,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91790,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,9,0.632517,0.482991,-1.874786e-07,...,0.484490,0.289181,-0.421546,0.572918,0.474172,-0.078406,0.397896,0.484275,-0.113990,1
91791,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,10,0.634008,0.486989,-1.780656e-07,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,1
91792,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,10,0.403709,0.829567,1.847001e-08,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,0
91793,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,11,0.643907,0.492386,-1.789463e-07,...,0.480457,0.287692,-0.410172,0.570401,0.466859,-0.068966,0.398920,0.484171,-0.099403,1


In [38]:
PH1_final_df.to_csv(sup.PH1_DATA_AH_PF_CSV, index=False)